In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/4_Compartment_Analysis_2021/Expt1.3_Eigenvector Weights/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/4_Compartment_Analysis_2021/Expt1.3_Eigenvector Weights/Output')
cur_date = "042121"

library(readxl)
library(data.table)
library(factoextra)
library(dplyr)
library(tidyverse)
library(gridExtra)
library(cluster)
library(vegan)
library(fpc)
library(ggdendro)
library(NbClust)
library(ggrepel)


#reading in file
cytokines <- data.frame(read_excel("CytokineData_032521.xlsx", sheet = 2))

Loading required package: ggplot2

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ tibble  3.0.6     ✔ purrr   0.3.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()


Attaching package: ‘gridExtra’


The f

Code to obtain weights to figure out the degree to which each cytokine contributed to each eigenvector. 

# Validation Dataset Weights

In [2]:
new_dataset_only_cytokines = c("GMCSF", "IL12p40", "IL15", "IL16", "IL17","IL1a", "IL2", "IL5", "IL7", "MCP4", "MDC","TNFb", "VEGF")
`%notin%` <- Negate(`%in%`)
#changing format of df
cytokines = cytokines %>% 
    #only want non-smokers for baseline analysis & cytokines that were in the previous dataset
    filter(Group == 'NS', Protein %notin% new_dataset_only_cytokines)
cytokines = reshape2::dcast(cytokines, SubjectID ~ Protein, value.var = "Conc_pslog2") %>% 
    column_to_rownames("SubjectID")
#transposed to reduce dimensionality or variation of subjects into principal components
#allows us to determine what cytokines are driving clusters
tcytokines = data.frame(t(cytokines))
head(tcytokines)

,NS_1,NS_10,NS_11,NS_12,NS_13,NS_14,NS_15,NS_16,NS_17,NS_18,NS_19,NS_2,NS_20,NS_3,NS_4,NS_5,NS_6,NS_7,NS_8,NS_9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Eotaxin,6.7747713,7.961318,5.798693,5.606658,6.852042,9.230327,5.6911455,7.950870,7.570201,5.349512,5.8250758,7.1852235,8.714178,8.840770,7.412499,7.258541,6.756939,7.605079,7.523347,6.0217081
Eotaxin3,7.3359976,3.561784,4.237103,3.133612,5.164080,7.794502,2.0189791,6.692010,5.525843,1.151684,5.2783081,4.0559827,6.266462,11.511480,7.519546,2.954984,3.981245,6.156015,4.261894,3.3244167
IFNg,2.7048125,1.501174,4.414765,2.708182,3.038039,3.233542,1.8037654,3.695319,3.510235,1.645145,1.3414476,6.1204987,3.396166,5.172665,3.824419,2.577966,2.973109,1.790451,3.597735,0.0000000
IL10,0.8786819,2.521602,3.887937,3.129121,3.572621,3.817435,1.9282279,3.902469,3.075462,1.025655,2.2546730,2.5890769,3.871091,6.251246,4.898731,2.924778,3.748700,2.062107,3.695351,2.1734049
IL12p70,0.0000000,1.432708,4.383899,2.299339,1.830181,2.661157,0.6751787,3.400877,1.431968,1.213433,0.4728952,0.9169927,2.473645,4.267356,3.706824,3.174891,2.489741,1.497252,2.911923,0.8209459
IL13,4.7858055,5.829412,7.105202,6.466481,6.184908,6.419010,4.4392043,7.017208,7.044898,4.762298,5.5844884,5.0105656,6.476432,6.860143,6.608278,5.209837,6.016760,6.044381,5.992372,5.3379432


In [3]:
#obtaining clusters from kmeans to get cluster assignments
#previously decided on 3 clusters
k3_tcytokines = kmeans(tcytokines, centers = 3, nstart = 25)
clusters_tcytokines = data.frame(k3_tcytokines$cluster) %>%
    rename("Cluster" = "k3_tcytokines.cluster") 
head(clusters_tcytokines)

,Cluster
,<int>
Eotaxin,1
Eotaxin3,1
IFNg,3
IL10,3
IL12p70,3
IL13,1


In [4]:
#now combining into 1 df
full_cytokine_df = data.frame(tcytokines, clusters_tcytokines) %>%
    #doing this so these row names are preserved when splitting df
    rownames_to_column(var = "Protein") %>%
    group_by(Cluster) %>%
    group_split

#obtaining each df after splitting
get_clustered_df = function(input_df, cluster_number){
    output_df = input_df[[cluster_number]] %>%
        column_to_rownames(var = "Protein") %>%
        select(-Cluster)
    return(output_df)
}

Cluster1_df = get_clustered_df(full_cytokine_df, 1)
Cluster2_df = get_clustered_df(full_cytokine_df, 2)
Cluster3_df = get_clustered_df(full_cytokine_df, 3)
head(Cluster3_df)

,NS_1,NS_10,NS_11,NS_12,NS_13,NS_14,NS_15,NS_16,NS_17,NS_18,NS_19,NS_2,NS_20,NS_3,NS_4,NS_5,NS_6,NS_7,NS_8,NS_9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IFNg,2.7048125,1.5011739,4.414765,2.708182,3.038039,3.233542,1.8037654,3.695319,3.5102349,1.6451449,1.3414476,6.1204987,3.396166,5.172665,3.824419,2.577966,2.973109,1.790451,3.597735,0.0000000
IL10,0.8786819,2.5216022,3.887937,3.129121,3.572621,3.817435,1.9282279,3.902469,3.0754619,1.0256545,2.2546730,2.5890769,3.871091,6.251246,4.898731,2.924778,3.748700,2.062107,3.695351,2.1734049
IL12p70,0.0000000,1.4327078,4.383899,2.299339,1.830181,2.661157,0.6751787,3.400877,1.4319676,1.2134332,0.4728952,0.9169927,2.473645,4.267356,3.706824,3.174891,2.489741,1.497252,2.911923,0.8209459
IL4,0.0498162,0.6891918,2.010562,1.800900,1.521769,1.887593,1.0879398,2.011810,0.8184517,1.2113780,1.2372493,0.6720234,1.722394,4.255033,2.804611,1.005298,1.587674,1.140736,2.249596,0.0000000
IL6,2.4927671,3.2238904,5.776803,5.191662,4.038158,3.796112,3.0139392,5.666587,3.4978949,1.3068766,2.4700822,3.1291245,4.478615,6.236309,5.112279,4.783985,3.724903,2.290082,3.870826,3.5220987
TNFa,1.3341330,1.8247650,2.910183,2.917108,2.300050,2.961765,0.9947875,3.727618,2.0256400,0.6871285,2.7863809,1.8192470,2.817299,5.669074,4.326751,1.964931,3.025899,1.012578,2.506878,2.0582313


In [5]:
#now we can run PCA on each
get_eigenvector_weights = function(df, cluster_name){
    pca = prcomp(df, scale = TRUE, center = TRUE)
    #x = values of each sample in terms of the principal components (ie. each cytokine's contribution to PC)
    #rotation = The relationship (correlation or anticorrelation, etc) between the initial variables and PCs
    pca_weights = data.frame(pca$x) %>% 
        #only returning first PC
        select(PC1) %>%
        plyr::rename(replace = c("PC1" = cluster_name)) %>%
        #this will make them easier to combine
        rownames_to_column(var = "Protein") %>%
        reshape2::melt(variable.name = "Cluster", value.name = "Weight")
        
            
    return(pca_weights)
}

#these correspond to previously established clusters
weights1_df = get_eigenvector_weights(Cluster1_df, "Cluster C") 
weights2_df = get_eigenvector_weights(Cluster2_df, "Cluster A")
weights3_df = get_eigenvector_weights(Cluster3_df, "Cluster B")
weights_df = rbind(weights1_df, weights2_df, weights3_df)
#adding a column to specify that this is from the validation dataset
weights_df$Dataset = rep("Validation", times = length(weights_df$Protein))
head(weights_df)

Using Protein as id variables

Using Protein as id variables

Using Protein as id variables



,Protein,Cluster,Weight,Dataset
,<chr>,<fct>,<dbl>,<chr>
1,Eotaxin,Cluster C,-2.8002327,Validation
2,Eotaxin3,Cluster C,3.8300461,Validation
3,IL13,Cluster C,0.5233879,Validation
4,IL1b,Cluster C,1.4680325,Validation
5,MCP1,Cluster C,-6.5713992,Validation
6,MIP1a,Cluster C,1.4779436,Validation


# Original Dataset Weights

In [12]:
#reading in file 
cytokines <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

#transposing df first
cytokines = cytokines %>% 
    #only want non-smokers for baseline analysis
    filter(Group == 'NS') %>%
    group_by(Compartment) %>%
    group_split

reshape_cytokines = function(df, number){
    new_df = reshape2::dcast(df[[number]], SubjectID ~ Protein, value.var = "Conc_pslog2") %>% 
    column_to_rownames("SubjectID")
    new_df = data.frame(t(new_df))
    return(new_df)
}

tNLF = reshape_cytokines(cytokines, 2)
tNELF = reshape_cytokines(cytokines, 1)
tSputum = reshape_cytokines(cytokines, 4)
tSerum = reshape_cytokines(cytokines, 3)
head(tNLF)

,NS_1,NS_10,NS_11,NS_12,NS_13,NS_14,NS_2,NS_3,NS_4,NS_5,NS_6,NS_7,NS_8,NS_9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Eotaxin,3.6280396,6.8187682,3.33682808,5.0333418,6.1047951,5.7762171,0.0000000,4.2097320,4.9384614,4.5968318,3.0585166,5.6634779,6.609432,4.90597963
Eotaxin3,3.1961268,7.1675129,3.95125524,2.1919254,6.2414741,3.7784170,3.2096483,5.0682269,3.5987808,2.6011813,1.3244504,5.0072047,9.407002,2.62280152
Fractalkine,11.5876883,11.8757287,9.22145132,11.5378449,11.5230939,11.2549039,7.8117047,10.0374539,10.6850491,11.0118375,10.7846908,11.4959189,11.509636,11.37872980
I309,0.0000000,0.0000000,0.00000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.000000,0.00000000
IFNg,0.2638601,0.4323116,0.24860903,1.8025665,1.7756286,1.5545991,0.0000000,0.1494616,0.8951656,5.8186862,0.3636737,0.8036332,1.480751,0.01754746
IL10,0.1734701,0.1682226,0.08642643,0.3754869,0.3477679,0.5740857,0.2434195,0.1561912,0.1550925,0.3022863,0.1194807,0.1833912,1.145162,0.06902558


In [14]:
#obtaining clusters from kmeans to get cluster assignments
#previously decided on 3 clusters
get_clusters = function(df){
    k3_tcytokines = kmeans(df, centers = 3, nstart = 25)
    clusters_tcytokines = data.frame(k3_tcytokines$cluster) %>%
        rename("Cluster" = "k3_tcytokines.cluster") 
    return(clusters_tcytokines)
}
clusters_tNLF = get_clusters(tNLF)
clusters_tNELF = get_clusters(tNELF)
clusters_tSputum = get_clusters(tSputum)
clusters_tSerum = get_clusters(tSerum)
head(clusters_tNLF)

,Cluster
,<int>
Eotaxin,1
Eotaxin3,1
Fractalkine,3
I309,2
IFNg,2
IL10,2
